In [217]:
import pandas as pd

In [218]:
# data processing
full_df = pd.read_csv("/home/jiwon/jiwon/pygbased/data/ML1M/raw/u.data", sep = '\t', names = ['user', 'item', 'rating', 'time'])
full_df.sort_values(by = 'time', inplace= True)

In [219]:
full_df.user = full_df.user - 1
full_df.item = full_df.item - 1
print(f"""{full_df.item.unique().min()}
{full_df.user.unique().min()}""")

0
0


In [220]:
def filter_core(df, user_col='user', item_col='item', k=20):
    """
    Filter the DataFrame to keep only interactions where each user and each item
    has at least 20 interactions.
    """
    # Repeat until the minimum interaction threshold is met for all users and items
    while True:
        # Count the number of interactions for each user and item
        user_interactions = df[user_col].value_counts()
        item_interactions = df[item_col].value_counts()
        
        # Identify users and items with fewer than 20 interactions
        few_user_interactions = user_interactions[user_interactions < k].index.tolist()
        few_item_interactions = item_interactions[item_interactions < k].index.tolist()
        
        # If all users and items have at least 20 interactions, stop
        if len(few_user_interactions) == 0 and len(few_item_interactions) == 0:
            break
        
        # Otherwise, remove users and items with too few interactions
        df = df[~df[user_col].isin(few_user_interactions)]
        df = df[~df[item_col].isin(few_item_interactions)]
    
    return df

filtered = filter_core(full_df, k = 10)

In [221]:
def print_static(df):
    n_u = df['user'].nunique()
    n_i = df['item'].nunique()
    n_interaction = df.shape[0]
    avg_u_d = n_interaction/n_u
    density = n_interaction / (n_u*n_i)
    time_start = df['time'].iloc[0]
    time_end = df['time'].iloc[-1]
    
    print(f"user:{n_u}")
    print(f"item:{n_i}")
    print(f"interaction:{n_interaction}")
    print(f"density:{density}")
    print(f"time start:{time_start}")
    print(f"time end:{time_end}")

print_static(filtered)

user:943
item:1152
interaction:97953
density:0.09016823524213503
time start:874724710
time end:893286638


In [222]:
from math import floor
def split_df(df, base_size ,inc_step):
    ratio = [base_size]
    ratio += [(1-base_size)/inc_step]*inc_step
    total_len = len(df)
    start=0
    result=[]
    
    for r in ratio:
        end=start+floor(total_len*r)
        result.append(df.iloc[start:end].reset_index(drop=True))
        start=end
    return result

In [223]:
splitted_ml = split_df(filtered,0.5,5)
splitted_ml

[       user  item  rating       time
 0       258   254       4  874724710
 1       258   285       4  874724727
 2       258   297       4  874724754
 3       258   184       4  874724781
 4       258   172       4  874724843
 ...     ...   ...     ...        ...
 48971   767  1013       2  882816126
 48972   828   249       3  882816754
 48973   828   221       4  882816987
 48974   667   895       4  882818549
 48975   667   287       4  882818604
 
 [48976 rows x 4 columns],
       user  item  rating       time
 0      454   297       4  882818787
 1       84   418       5  882819427
 2       84   160       4  882819528
 3       28   301       4  882820663
 4       28   285       5  882820663
 ...    ...   ...     ...        ...
 9790   523   572       4  884636827
 9791   523   435       4  884636864
 9792   523   229       3  884636907
 9793   523   525       3  884636907
 9794   523   548       4  884636931
 
 [9795 rows x 4 columns],
       user  item  rating       time
 0    

In [224]:
from torch_geometric.data import HeteroData
import torch
import numpy as np

class DataInformation(object):
    def __init__(self, total_df, user_id_mapping, item_id_mapping, id_user_mapping, id_item_mapping, user_consumed, item_consumed, n_user, n_item, user_unique_value,item_unique_value, seed):
        self.total_df = total_df
        self.user_id_mapping = user_id_mapping
        self.item_id_mapping = item_id_mapping

        self.id_user_mapping = id_user_mapping
        self.id_item_mapping = id_item_mapping

        self.user_consumed = user_consumed
        self.item_consumed = item_consumed
        
        self.n_user = n_user
        self.n_item = n_item

        self.user_unique_value = user_unique_value
        self.item_unique_value = item_unique_value

        self.seed = seed


class Continual_Data(object):
    def make_dataset(cls, train_df, val_df, test_df, seed):
        """
        TODO:: 
        기존 데이터 중 user-id mapping : 기존 유저아이디를 1부터 해서 점점 증가시킴
        기존 데이터 중 item-id mapping : 기존 item아이디를 1부터 해서 점점 증가시킴
        consume-set이라고 기존 유저가 소비한 아이템과 아이템을 소비한 유저 매핑 만들기
        생성해서 data_info에 집어넣기
        최종적으로 pytorch-graph형태의 데이터셋 생성
        """
        total_df = pd.concat([train_df, val_df, test_df])
        # 유저 아이디 매핑 
        user_set = set(total_df['user'].unique())
        item_set = set(total_df['item'].unique())

        n_user = len(user_set)
        n_item = len(item_set)

        user_id_mapping = {user: id for user, id in zip(user_set, range(n_user))}
        item_id_mapping = {item: id for item, id in zip(item_set, range(n_item))}
        id_user_mapping = {id: user for user, id in user_id_mapping.items()}
        id_item_mapping = {id: item for item, id in item_id_mapping.items()}

        """
        TODO:: 만약 train에 없는 유저 혹은 아이템이 val 혹은 test에 등장할경우 제거해주는 코드 작성하기
        """
        


        train_df['user'] = train_df['user'].apply(lambda x: user_id_mapping[x])
        train_df['item'] = train_df['item'].apply(lambda x: item_id_mapping[x])
        val_df['user'] = val_df['user'].apply(lambda x: user_id_mapping[x])
        val_df['item'] = val_df['item'].apply(lambda x: item_id_mapping[x])
        test_df['user'] = test_df['user'].apply(lambda x: user_id_mapping[x])
        test_df['item'] = test_df['item'].apply(lambda x: item_id_mapping[x])


        user_consumed = train_df.groupby('user')['item'].apply(list).reset_index(name="consumed_items")
        item_consumed = train_df.groupby('item')['user'].apply(list).reset_index(name="consumed_users")

        user_consumed = user_consumed.to_dict('list')
        item_consumed = item_consumed.to_dict('list')


        # Make pyg data
        data = HeteroData()
        data['user'].num_nodes = n_user
        data['item'].num_nodes = n_item

        attr_names = ['edge_train_index', 'edge_val_index', 'edge_test_index', 'edge_total_index']
        dataframes = [train_df, val_df, test_df, train_df]
        
        answer_set = {}

        for attr_name, temp_df in zip(attr_names, dataframes):
            rows = temp_df['user'].values
            cols = temp_df['item'].values
            index = torch.tensor([rows, cols])

            answer_set[attr_name] = index
            
            data['user', 'rates', 'item'][attr_name] = index
            if attr_name == 'edge_train_index':
                data['item', 'rated_by', 'user'][attr_name] = index.flip([0])


        return data.to_homogeneous(),answer_set , DataInformation(total_df, user_id_mapping, item_id_mapping, id_user_mapping, id_item_mapping, user_consumed, item_consumed, n_user, n_item, user_set, item_set, seed)

        
    def merge_dataset(cls, train_df, val_df, test_df ,prev_info: DataInformation):
        """
        TODO::
        기존데이터에 id 매핑 뒤쪽에 추가시키고 나머지 데이터 정보도 업데이트 시키기
        최종적으로 pytorch-graph형태의 데이터셋 생성
        """

        total_df = pd.concat([train_df, val_df, test_df])

        new_user_set = set(total_df['user'].unique())
        new_item_set = set(total_df['item'].unique())

        total_user_set = prev_info.user_unique_value | new_user_set
        total_item_set = prev_info.item_unique_value | new_item_set

        new_users = new_user_set.difference(prev_info.user_unique_value)
        new_items = new_item_set.difference(prev_info.item_unique_value)

        new_n_user = prev_info.n_user + len(new_users)
        new_n_item = prev_info.n_item + len(new_items)
        

        new_user_id_mapping = {user: id for user, id in zip(new_user_set, range(prev_info.n_user+new_n_user))}
        new_item_id_mapping = {item: id for item, id in zip(new_item_set, range(prev_info.n_item+new_n_item))}
        new_id_user_mapping = {id: user for user, id in new_user_id_mapping.items()}
        new_id_item_mapping = {id: item for item, id in new_item_id_mapping.items()}

        total_user_id_mapping = prev_info.user_id_mapping
        total_item_id_mapping = prev_info.item_id_mapping

        total_id_user_mapping = prev_info.id_user_mapping
        total_id_item_mapping = prev_info.id_item_mapping

        total_user_id_mapping.update(new_user_id_mapping)
        total_item_id_mapping.update(new_item_id_mapping)

        total_id_user_mapping.update(new_id_user_mapping)
        total_id_item_mapping.update(new_id_item_mapping)
        """
        TODO:: 만약 train에 없는 유저 혹은 아이템이 val 혹은 test에 등장할경우 제거해주는 코드 작성하기
        """

        train_df['user'] = train_df['user'].apply(lambda x: total_user_id_mapping[x])
        train_df['item'] = train_df['item'].apply(lambda x: total_item_id_mapping[x])
        val_df['user'] = val_df['user'].apply(lambda x: total_user_id_mapping[x])
        val_df['item'] = val_df['item'].apply(lambda x: total_item_id_mapping[x])
        test_df['user'] = test_df['user'].apply(lambda x: total_user_id_mapping[x])
        test_df['item'] = test_df['item'].apply(lambda x: total_item_id_mapping[x])

        new_user_consumed = train_df.groupby('user')['item'].apply(list).reset_index(name="consumed_items")
        new_item_consumed = train_df.groupby('item')['user'].apply(list).reset_index(name="consumed_users")

        new_user_consumed = new_user_consumed.to_dict('list')
        new_item_consumed = new_item_consumed.to_dict('list')

        total_user_consumed = prev_info.user_consumed
        total_item_consumed = prev_info.item_consumed

        for key, val in new_user_consumed.items():
            if key in total_user_consumed.keys():
                total_user_consumed[key].extend(val)
            else:
                total_user_consumed[key] = val
        
        for key, val in new_item_consumed.items():
            if key in total_item_consumed.keys():
                total_item_consumed[key].extend(val)
            else:
                total_item_consumed[key] = val

        # Make pyg data
        data = HeteroData()
        data['user'].num_nodes = new_n_user
        data['item'].num_nodes = new_n_item

        total_df = pd.concat([prev_info.total_df, train_df])
        attr_names = ['edge_train_index', 'edge_val_index', 'edge_test_index', 'edge_total_index']
        dataframes = [train_df, val_df, test_df, total_df]

        answer_set = {}

        for attr_name, temp_df in zip(attr_names, dataframes):
            rows = temp_df['user'].values
            cols = temp_df['item'].values
            index = torch.tensor([rows, cols])
            
            answer_set[attr_name] = index

            data['user', 'rates', 'item'][attr_name] = index

            if attr_name == 'edge_train_index':
                data['item', 'rated_by', 'user'][attr_name] = index.flip([0])

        new_total_df = pd.concat([total_df, val_df, test_df])

        return data.to_homogeneous(),answer_set ,DataInformation(new_total_df, total_user_id_mapping, total_item_id_mapping, total_id_user_mapping, total_id_item_mapping, total_user_consumed, total_item_consumed, new_n_user, new_n_item, total_user_set, total_item_set, prev_info.seed)

    def save_data_info(self):
        """
        TODO:: 
        저장해야할 Variable들
        user-id mapping
        item-id mapping
        user-consumeset
        item-consumedset
        n_user
        n_item
        user_unique_value
        item_unique_value
        seed
        """


In [251]:
import torch_geometric as pyg
from torch_geometric.nn import LightGCN
import torch.nn as nn
import torch

class ContinualModel(nn.Module):
    def __init__(self, data, embedding_dim, device):
        super().__init__()
        # self.inc = inc
        self.data = data
        self.embedding_dim = embedding_dim
        self.device = device
        self.model = self.build_model(data)

    def forward(self, index, train_edge_label_index):

        pos_rank, neg_rank = self.model(index, train_edge_label_index).chunk(2)

        loss = self.model.recommendation_loss(
            pos_rank,
            neg_rank,
            node_id=train_edge_label_index.unique(),
            lambda_reg = 0.0001
        )

        # 추가적인 로스는 여기서 계산

        return loss , pos_rank

    def save_model(self):
        """
        TODO:: embedding 저장하기
        """

    def build_model(self, data):
        
        return LightGCN(
            num_nodes=data.num_nodes,
            embedding_dim=self.embedding_dim,
            num_layers=3
        ).to(self.device)
    
    @torch.no_grad
    def rebuild_model(self, emb, prev_num_user, prev_num_item, num_user, num_item):
        # 기존 모델에서 임베딩 가져옴 -> 유저와 아이템 기존의 갯수 만큼 나눔
        prev_model = 0 # 기존 모델 불러오기
        prev_emb = emb
        prev_user_emb, prev_item_emb = prev_emb[:prev_num_user], prev_emb[prev_num_user:]

        avg_user_emb = torch.mean(prev_user_emb, dim = 0).repeat(num_user-prev_num_user,1)
        avg_item_emb = torch.mean(prev_item_emb, dim = 0).repeat(num_item-prev_num_item,1)



        new_user_emb = torch.stack([prev_user_emb, avg_user_emb])
        new_item_emb = torch.stack([prev_item_emb, avg_item_emb])
        
        new_total_emb = torch.stack([new_user_emb, new_item_emb])

        self.model.embedding.data = new_total_emb.data





In [252]:
import os.path as osp
import os

#torch 전에 할댕해줘야함
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]= "1"


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device:', device)
print('Current cuda device:', torch.cuda.current_device())
print('Count of using GPUs:', torch.cuda.device_count())


import torch
from tqdm import tqdm

from torch_geometric.datasets import AmazonBook, MovieLens100K
from torch_geometric.nn import LightGCN
from torch_geometric.utils import degree
from torch_geometric.data import InMemoryDataset

Device: cuda
Current cuda device: 0
Count of using GPUs: 1


In [253]:
@torch.no_grad()
def evaluate(model, data, answer_set, data_info, k: int, batch_size, device):
    emb = model.model.get_embedding(data.edge_train_index)
    user_emb, book_emb = emb[:data_info.n_user], emb[data_info.n_user:]

    precision = recall = total_examples = 0
    for start in range(0, data_info.n_user, batch_size):
        end = min(start + batch_size, data_info.n_user)
        logits = user_emb[start:end] @ book_emb.t()
        # # Exclude training edges:
        # mask = ((train_edge_label_index[0] >= start) &
        #         (train_edge_label_index[0] < end))
        # logits[train_edge_label_index[0, mask] - start,
        #         train_edge_label_index[1, mask] - data_info.n_user] = float('-inf')
        # Exclude total edges:
        mask = ((answer_set['edge_total_index'] >= start) &
                (answer_set['edge_total_index'] < end))
        logits[answer_set['edge_total_index'][mask] - start,
                answer_set['edge_total_index'][mask] - data_info.n_user] = float('-inf')

        # Computing precision and recall:
        ground_truth = torch.zeros_like(logits, dtype=torch.bool)
        mask = ((answer_set['edge_val_index'][0] >= start) &
                (answer_set['edge_val_index'][0] < end))
        ground_truth[answer_set['edge_val_index'][0, mask] - start,
                        answer_set['edge_val_index'][1, mask] - data_info.n_user] = True
        node_count = degree(answer_set['edge_val_index'][0, mask] - start,
                                num_nodes=logits.size(0)).to(device)

        topk_index = logits.topk(k, dim=-1).indices
        isin_mat = ground_truth.gather(1, topk_index)

        precision += float((isin_mat.sum(dim=-1) / k).sum())
        recall += float((isin_mat.sum(dim=-1) / node_count.clamp(1e-6)).sum())
        total_examples += int((node_count > 0).sum())

    return precision / total_examples, recall / total_examples

In [254]:
from sklearn.model_selection import train_test_split
data = Continual_Data()

best_emb = [0]*len(splitted_ml)
info = []
for i in range(len(splitted_ml)-1):
    if i == 0:
        train, val = train_test_split(splitted_ml[i], test_size=0.2)
        val, test = train_test_split(val, test_size=0.5)
        graph_data, answer_set, data_info = data.make_dataset(train, val, test, 42)
        info.append(data_info)
    else:
        train = splitted_ml[i]
        val, test = train_test_split(splitted_ml[i+1], test_size=0.5)
        graph_data, answer_set, data_info = data.merge_dataset(train, val, test, data_info)
        info.append(data_info)
    graph_data.to(device)
    batch_size = 1024
    mask = graph_data.edge_train_index[0] < graph_data.edge_train_index[1]
    train_edge_label_index = graph_data.edge_train_index[:, mask]
    train_loader = torch.utils.data.DataLoader(
        range(train_edge_label_index.size(1)),
        shuffle=True,
        batch_size=batch_size,
    )

    model = ContinualModel(graph_data, 64, device)

    """
    TODO::
    여기서 기존의 모델 가지고 온다음에 rebuild 수행 할 것   
    """
    if i != 0:
        model.rebuild_model(best_emb[i-1], info[i-1].n_user, info[i-1].n_item, data_info.n_user, data_info.n_item)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    best_recall = 0
    with tqdm(range(100)) as tepoch:

        for epoch in tepoch:
            
            train_loader = torch.utils.data.DataLoader(
            range(train_edge_label_index.size(1)),
            shuffle=True,
            batch_size=batch_size,
            )   
            

            total_loss = total_examples = 0

            for index in train_loader:
                # Sample positive and negative labels.
                pos_edge_label_index = train_edge_label_index[:, index]
                neg_edge_label_index = torch.stack([
                    pos_edge_label_index[0],
                    torch.randint(data_info.n_user, data_info.n_user + data_info.n_item,
                                (index.numel(), ), device=device)
                ], dim=0)
                edge_label_index = torch.cat([
                    pos_edge_label_index,
                    neg_edge_label_index,
                ], dim=1)

                optimizer.zero_grad()
                loss, pos_rank = model(graph_data.edge_train_index, edge_label_index)

                loss.backward()
                optimizer.step()

                total_loss += float(loss) * pos_rank.numel()
                total_examples += pos_rank.numel()
            recall, precision = evaluate(model, graph_data, answer_set, data_info,20, batch_size, device)
            if recall > best_recall:
                best_recall = recall
                best_emb[i] = model.model.embedding.weight.detach()
            tepoch.set_postfix(loss=total_loss / total_examples, recall = recall, precision = precision)
        

100%|██████████| 100/100 [00:14<00:00,  7.13it/s, loss=0.0926, precision=0.0452, recall=0.0213]


RuntimeError: stack expects each tensor to be equal size, but got [489, 64] at entry 0 and [119, 64] at entry 1

In [52]:
splitted_ml[i]

,user,item,rating,time
0,258,254,4,874724710
1,258,285,4,874724727
2,258,297,4,874724754
3,258,184,4,874724781
4,258,172,4,874724843
...,...,...,...,...
48971,767,1013,2,882816126
48972,828,249,3,882816754
48973,828,221,4,882816987
48974,667,895,4,882818549
